In [126]:
import pandas as pd
from sqlalchemy import create_engine

# Load datasets

### After downloading the two separate datasets (Census and Zillow) in csv format from Kaggle, loading these into Jupyter Notebook in order to start the cleaning process and next merging the data now in dataframe format.

In [127]:
#loading first dataset
census_file = r"C:\Users\clara\Documents\GitHub\Project 2\Census_dataset\acs2015_county_data.csv"
census_df = pd.read_csv(census_file)
census_df.head()

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,...,0.5,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6
1,1003,Alabama,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,...,1.0,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5
2,1005,Alabama,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,...,1.8,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6
3,1007,Alabama,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,...,0.6,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3
4,1009,Alabama,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,...,0.9,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7


In [128]:
#Loading second dataset
zillow_file = r"C:\Users\clara\Documents\GitHub\Project 2\Zillow_dataset\price.csv"
zillow_df = pd.read_csv(zillow_file)
zillow_df.head()

,City Code,City,Metro,County,State,Population Rank,November 2010,December 2010,January 2011,February 2011,...,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017
0,6181,New York,New York,Queens,NY,1,NaN,NaN,NaN,NaN,...,2334,2339,2345,2344,2336,2324,2318,2321,2321,2322
1,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,2184.0,2184.0,2183.0,2188.0,...,2637,2662,2687,2704,2716,2723,2731,2740,2748,2753
2,17426,Chicago,Chicago,Cook,IL,3,1563.0,1555.0,1547.0,1537.0,...,1684,1686,1687,1685,1681,1675,1668,1656,1644,1632
3,39051,Houston,Houston,Harris,TX,4,1198.0,1199.0,1199.0,1200.0,...,1444,1446,1446,1443,1440,1438,1437,1437,1435,1430
4,13271,Philadelphia,Philadelphia,Philadelphia,PA,5,1092.0,1099.0,1094.0,1087.0,...,1206,1211,1218,1222,1223,1220,1216,1211,1209,1212


In [129]:
#Verifying columns in the Census dataset
census_df.columns

Index(['CensusId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen', 'Income',
       'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

# Clean dataframe


### Multiple steps were taken to clean each dataframe explained in the cells below. Each dataframe had to be modified, cleaned and/or renamed in order to achieve one column with matching results to be used as the key for the merge. In this case, the column "County" in each dataframe was utilized, but had to be previously adjusted

In [130]:
#Creating new Zillow dataframe with select columns
#Filtering Zillow dataset to keep only 2015 values to match the 2015 Census dataset. Originally the Zillow dataset contained data on the year 2010 to 2017
zillow2015_df = zillow_df[['City Code', 'City', 'Metro', 'County', 'State', 'Population Rank', 'January 2015', 'February 2015',
       'March 2015', 'April 2015', 'May 2015', 'June 2015', 'July 2015',
       'August 2015', 'September 2015', 'October 2015', 'November 2015',
       'December 2015']]
zillow2015_df.head()

,City Code,City,Metro,County,State,Population Rank,January 2015,February 2015,March 2015,April 2015,May 2015,June 2015,July 2015,August 2015,September 2015,October 2015,November 2015,December 2015
0,6181,New York,New York,Queens,NY,1,2216,2229,2241,2248,2253,2251,2246,2259,2276,2304,2322,2334
1,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,2445,2464,2479,2493,2502,2511,2521,2536,2546,2555,2564,2577
2,17426,Chicago,Chicago,Cook,IL,3,1677,1668,1668,1669,1667,1668,1670,1668,1660,1652,1649,1653
3,39051,Houston,Houston,Harris,TX,4,1377,1384,1389,1394,1402,1410,1419,1425,1428,1428,1429,1431
4,13271,Philadelphia,Philadelphia,Philadelphia,PA,5,1182,1185,1183,1178,1176,1179,1179,1177,1175,1179,1184,1189


In [131]:
#Double checking repeated county rows for the zillow dataset. County column will be used as key for merge
# zillow2015_df.count()
# len(zillow2015_df)
zillow2015_df.nunique()

City Code          13131
City                9670
Metro                701
County              1227
State                 51
Population Rank    13131
January 2015        2296
February 2015       2313
March 2015          2300
April 2015          2331
May 2015            2347
June 2015           2348
July 2015           2331
August 2015         2346
September 2015      2352
October 2015        2374
November 2015       2377
December 2015       2384
dtype: int64

In [132]:
#Double checking repeated county rows for the census dataset. County column will be used as foreign key for merge
#This step identified that several County rows had similar names but in all likelihood different States, representing actually different counties
#For that reason it was required to use two different columns as keys, County and State
#This step ensured that counties named similarly but in different states were treated as unique and not duplicates 
# census_df.count()
# len(census_df)
census_df.nunique()

CensusId           3220
State                52
County             1928
TotalPop           3164
Men                3098
Women              3105
Hispanic            466
White               721
Black               487
Native              183
Asian               131
Pacific              27
Citizen            3146
Income             3066
IncomeErr          2329
IncomePerCap       2962
IncomePerCapErr    1758
Poverty             394
ChildPoverty        544
Professional        344
Service             228
Office              202
Construction        237
Production          292
Drive               365
Carpool             189
Transit             119
Walk                171
OtherTransp          93
WorkAtHome          188
MeanCommute         309
Employed           3098
PrivateWork         381
PublicWork          322
SelfEmployed        221
FamilyWork           39
Unemployment        230
dtype: int64

In [133]:
#Renaming the "State" column in census_df to match the State column in the zillow_df, since State will also be used as a key
#This is required in order to identify the correct county per state, given that there are similarly named counties in different states causing false duplicates
census_df.State = census_df.State.map({
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
})
census_df.head()

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,AL,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,...,0.5,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6
1,1003,AL,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,...,1.0,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5
2,1005,AL,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,...,1.8,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6
3,1007,AL,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,...,0.6,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3
4,1009,AL,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,...,0.9,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7


In [134]:
#Double checking States renaming was successful
census_df.State.unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR'], dtype=object)

In [135]:
#Merging the two dataframes, using the Zillow as the left on a left join since the Zillow dataframe had less counties listed than the Census dataframe
#Using the Zillow dataframe as the left allowed to avoid having multiple rows with missing data which would need to be cleaned up next
merged_data = zillow2015_df.merge(census_df, on=["State", "County"], how="left")
merged_data.head()

,City Code,City,Metro,County,State,Population Rank,January 2015,February 2015,March 2015,April 2015,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,6181,New York,New York,Queens,NY,1,2216,2229,2241,2248,...,5.8,1.5,2.4,42.6,1104930.0,79.9,13.6,6.4,0.1,8.6
1,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,2445,2464,2479,2493,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
2,17426,Chicago,Chicago,Cook,IL,3,1677,1668,1668,1669,...,4.4,2.2,4.2,32.3,2463655.0,83.9,11.5,4.5,0.1,10.7
3,39051,Houston,Houston,Harris,TX,4,1377,1384,1389,1394,...,1.5,2.0,3.3,28.2,2081889.0,83.4,10.1,6.3,0.1,7.5
4,13271,Philadelphia,Philadelphia,Philadelphia,PA,5,1182,1185,1183,1178,...,8.5,3.3,3.1,32.5,640661.0,82.9,13.0,4.0,0.1,13.9


In [136]:
#Checking for null values
#The Metro column was identified with multiple null values since it was only relevant for counties in Metro areas
#Metro column not required for purposes of this study as counties in reference can be identified by county, state and city
merged_data.isna().sum()

City Code             0
City                  0
Metro              1232
County                0
State                 0
Population Rank       0
January 2015          0
February 2015         0
March 2015            0
April 2015            0
May 2015              0
June 2015             0
July 2015             0
August 2015           0
September 2015        0
October 2015          0
November 2015         0
December 2015         0
CensusId            449
TotalPop            449
Men                 449
Women               449
Hispanic            449
White               449
Black               449
Native              449
Asian               449
Pacific             449
Citizen             449
Income              449
IncomeErr           449
IncomePerCap        449
IncomePerCapErr     449
Poverty             449
ChildPoverty        449
Professional        449
Service             449
Office              449
Construction        449
Production          449
Drive               449
Carpool         

In [137]:
#Deleting Metro column
#Identifying other rows with null values. 449 Counties flagged with missing data as a result of the merge
merged_data.drop(columns = "Metro", inplace = True)
merged_data.isna().sum()

City Code            0
City                 0
County               0
State                0
Population Rank      0
January 2015         0
February 2015        0
March 2015           0
April 2015           0
May 2015             0
June 2015            0
July 2015            0
August 2015          0
September 2015       0
October 2015         0
November 2015        0
December 2015        0
CensusId           449
TotalPop           449
Men                449
Women              449
Hispanic           449
White              449
Black              449
Native             449
Asian              449
Pacific            449
Citizen            449
Income             449
IncomeErr          449
IncomePerCap       449
IncomePerCapErr    449
Poverty            449
ChildPoverty       449
Professional       449
Service            449
Office             449
Construction       449
Production         449
Drive              449
Carpool            449
Transit            449
Walk               449
OtherTransp

In [138]:
#Showing lines with null values, issue identified with County naming
#Examples of County naming issues (mismatches from the Census df and Zillow df) identified: counties containing the word 'City' 
#as opposed to not containing that info (Baltimore City x Baltimore), counties containing the word 'Saint' as opposed to 'St'
#Made the decision to eliminate the 449 rows with county naming issues from a sample of 13131 counties
merged_data[merged_data.CensusId.isna()]

,City Code,City,County,State,Population Rank,January 2015,February 2015,March 2015,April 2015,May 2015,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
23,3523,Baltimore,Baltimore City,MD,24,1294,1292,1292,1296,1303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,41515,Virginia Beach,Virginia Beach City,VA,39,1534,1536,1537,1541,1546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,6891,Saint Louis,Saint Louis City,MO,57,844,848,849,849,849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,23482,Anchorage,Anchorage,AK,63,1958,1962,1976,1979,1978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,6210,Norfolk,Norfolk City,VA,84,1255,1260,1265,1266,1266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12988,14167,Tower,Saint Louis,MN,12989,1044,1059,1080,1094,1094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12990,15070,Callaway,Saint Marys,MD,12991,1469,1463,1465,1487,1502,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13019,24799,Freeburg,Saint Clair,IL,13020,1072,1056,1045,1044,1053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13093,49693,Piney Point,Saint Marys,MD,13094,1461,1444,1442,1451,1474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
#Droping remaining lines with null values
merged_data = merged_data.dropna()
merged_data

,City Code,City,County,State,Population Rank,January 2015,February 2015,March 2015,April 2015,May 2015,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,6181,New York,Queens,NY,1,2216,2229,2241,2248,2253,...,5.8,1.5,2.4,42.6,1104930.0,79.9,13.6,6.4,0.1,8.6
1,12447,Los Angeles,Los Angeles,CA,2,2445,2464,2479,2493,2502,...,2.8,2.3,5.1,30.0,4635465.0,79.0,11.5,9.4,0.2,10.0
2,17426,Chicago,Cook,IL,3,1677,1668,1668,1669,1667,...,4.4,2.2,4.2,32.3,2463655.0,83.9,11.5,4.5,0.1,10.7
3,39051,Houston,Harris,TX,4,1377,1384,1389,1394,1402,...,1.5,2.0,3.3,28.2,2081889.0,83.4,10.1,6.3,0.1,7.5
4,13271,Philadelphia,Philadelphia,PA,5,1182,1185,1183,1178,1176,...,8.5,3.3,3.1,32.5,640661.0,82.9,13.0,4.0,0.1,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,397405,Highland Township,Adams,PA,13127,1294,1286,1279,1295,1321,...,4.0,1.1,3.7,27.2,50517.0,82.9,11.1,5.9,0.2,5.7
13127,398292,Town of Wrightstown,Brown,WI,13128,1114,1117,1130,1138,1178,...,2.5,1.8,3.6,18.9,130354.0,84.6,11.0,4.3,0.1,6.1
13128,398343,Urbana,Steuben,NY,13129,1624,1618,1600,1587,1572,...,4.0,1.4,3.1,21.6,43199.0,76.4,16.7,6.7,0.3,8.2
13129,398839,Angels,Calaveras,CA,13130,1457,1463,1473,1463,1461,...,1.9,0.8,7.5,33.6,16248.0,65.2,20.3,13.9,0.6,12.1


In [140]:
#Renaming columns in the merged dataframe to eliminate spaces and facilitate creating table in SQL
merged_data.rename(columns={
    'City Code': 'City_code', 
    'Population Rank': 'Population_Rank',
    'January 2015': 'January_2015', 
    'February 2015': 'February_2015', 
    'March 2015': 'March_2015', 
    'April 2015': 'April_2015', 
    'May 2015': 'May_2015',
    'June 2015': 'June_2015', 
    'July 2015': 'July_2015', 
    'August 2015': 'August_2015', 
    'September 2015': 'September_2015',
    'October 2015': 'October_2015', 
    'November 2015': 'November_2015', 
    'December 2015': 'December_2015'
}, inplace=True)

In [142]:
#Getting correct spelling of columns and associated data types to copy into SQL for table creation
datatype = merged_data.dtypes
print(datatype)

City_code            int64
City                object
County              object
State               object
Population_Rank      int64
January_2015         int64
February_2015        int64
March_2015           int64
April_2015           int64
May_2015             int64
June_2015            int64
July_2015            int64
August_2015          int64
September_2015       int64
October_2015         int64
November_2015        int64
December_2015        int64
CensusId           float64
TotalPop           float64
Men                float64
Women              float64
Hispanic           float64
White              float64
Black              float64
Native             float64
Asian              float64
Pacific            float64
Citizen            float64
Income             float64
IncomeErr          float64
IncomePerCap       float64
IncomePerCapErr    float64
Poverty            float64
ChildPoverty       float64
Professional       float64
Service            float64
Office             float64
C

# Connect to local database

## Using Python to connect AWS RDS MySQL database and using the Create Engine from the SQLAlchemy library to interpret the database API

In [147]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'census_and_zillow2'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [148]:
engine.table_names()

['merged_data']

In [149]:
merged_data.to_sql(name='census_and_zillow', con=engine, if_exists='append', index=False)